In [11]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path

In [ ]:
import os

TumorPath = ".\\Tumor\\Brain Tumor"
HealthyPath = ".\\Tumor\\Healthy"
X = []

for r,ds,fs in os.walk(TumorPath):
    break
for image in fs:
    img = cv2.imread(TumorPath+"\\"+image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if img.shape[0] != 128 or img.shape[1] != 128:
        img = cv2.resize(img, dsize=(128,128))
    X.append(img)
y1 = np.ones((len(X),))
lastXlen = len(X)
for r,ds,fs in os.walk(HealthyPath):
    break
for image in fs:
    img = cv2.imread(HealthyPath+"\\"+image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if img.shape[0] != 128 or img.shape[1] != 128:
        img = cv2.resize(img, dsize=(128,128))
    X.append(img)
y2 = np.zeros((len(X)-lastXlen,))
X = np.array(X, dtype=np.float32)
y = np.concat([y1,y2])
print(X.shape)
print(y.shape)

(4606, 128, 128)
(4606,)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train = X_train.reshape(X_train.shape[0],128,128,1)
X_test = X_test.reshape(X_test.shape[0],128,128,1)

print("اندازه داده‌ها:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

اندازه داده‌ها:
X_train: (3684, 128, 128, 1)
X_test: (922, 128, 128, 1)
y_train: (3684,)
y_test: (922,)


In [14]:
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Add

output_activation = 'sigmoid'
loss_fn = 'binary_crossentropy'

def residual_block(x, filters):
    shortcut = x
    x = Conv2D(filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)

    # skip connection
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# ورودی
inputs = Input(shape=(128,128,1))

# Block 1
x = Conv2D(16, (3,3), padding='same', activation='relu')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

# Residual Block 1
x = residual_block(x, 16)

# Block 2
x = Conv2D(32, (3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

# Residual Block 2
x = residual_block(x, 32)

# Block 3
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

# Residual Block 3
x = residual_block(x, 64)

# Global pooling + Dense
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation=output_activation)(x)

# ساخت مدل
model = Model(inputs, outputs)

# کامپایل
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

# خلاصه مدل
model.summary()

# آموزش مدل
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=32
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 128, 128,  │        160 │ input_layer_1[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         64 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 64, 64,    │      2,320 │ max_pooling2d_3[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │         64 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 64, 64,    │      2,320 │ activation_6[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │         64 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 64, 64,    │          0 │ add_3[0][0]       │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 64, 64,    │      4,640 │ activation_7[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        128 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 32, 32,    │      9,248 │ max_pooling2d_4[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_13[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 32, 32,    │          0 │ batch_normalizat

 Total params: 130,081 (508.13 KB)

 Trainable params: 129,409 (505.50 KB)

 Non-trainable params: 672 (2.62 KB)

Epoch 1/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 30s 225ms/step - accuracy: 0.7182 - loss: 0.5457 - val_accuracy: 0.6236 - val_loss: 0.6385
Epoch 2/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 25s 220ms/step - accuracy: 0.8379 - loss: 0.3823 - val_accuracy: 0.5564 - val_loss: 1.2347
Epoch 3/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 26s 220ms/step - accuracy: 0.8757 - loss: 0.2869 - val_accuracy: 0.7798 - val_loss: 0.4555
Epoch 4/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 26s 220ms/step - accuracy: 0.9126 - loss: 0.2116 - val_accuracy: 0.7636 - val_loss: 0.4781
Epoch 5/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 26s 220ms/step - accuracy: 0.9368 - loss: 0.1788 - val_accuracy: 0.8948 - val_loss: 0.2988
Epoch 6/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 25s 219ms/step - accuracy: 0.9482 - loss: 0.1437 - val_accuracy: 0.9262 - val_loss: 0.1886
Epoch 7/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 26s 220ms/step - accuracy: 0.9503 - loss: 0.1353 - val_accuracy: 0.9501 - val_loss: 0.1409
Epoch 8/15
116/116 ━━━━━━━━━━━━━━━━━━━━ 26s 221ms/step - accuracy: 0.9612 - loss: 0

In [15]:
model.save('Tumor_ResNet.keras')